In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np

# Step 1: Define some real pages (small network for demonstration)
pages = {
    "https://www.python.org": [],
    "https://docs.python.org/3/": [],
    "https://pypi.org/": [],
    "https://www.djangoproject.com/": []
}


# Step 2: Extract all hyperlinks from each page
for page in pages.keys():
    try:
        print(f"Fetching links from: {page}")
        response = requests.get(page, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        links = [a['href'] for a in soup.find_all('a', href=True)]
        # Keep only internal links (for safety)
        links = [link for link in links if link.startswith("https://www.python.org") 
                 or link.startswith("https://docs.python.org")
                 or link.startswith("https://pypi.org")
                 or link.startswith("https://www.djangoproject.com")]
        pages[page] = links
    except Exception as e:
        print(f"Error fetching {page}: {e}")

# Step 3: Build the link matrix
n = len(pages)
page_list = list(pages.keys())
link_matrix = np.zeros((n, n))

for i, page in enumerate(page_list):
    links = pages[page]
    if len(links) > 0:
        for link in links:
            if link in page_list:
                j = page_list.index(link)
                link_matrix[j][i] = 1 / len(links)

# Step 4: Apply the PageRank formula
damping_factor = 0.85
num_iterations = 100
rank = np.ones(n) / n

for _ in range(num_iterations):
    rank = (1 - damping_factor) / n + damping_factor * link_matrix.dot(rank)

# Step 5: Display PageRank values
print("\nFinal PageRank Scores:")
for i, page in enumerate(page_list):
    print(f"{page}: {rank[i]:.4f}")

Fetching links from: https://www.python.org
Fetching links from: https://docs.python.org/3/
Fetching links from: https://pypi.org/
Fetching links from: https://www.djangoproject.com/

Final PageRank Scores:
https://www.python.org: 0.0375
https://docs.python.org/3/: 0.0375
https://pypi.org/: 0.0415
https://www.djangoproject.com/: 0.0385
